Сначала загрузим тексты и предобработаем (код из файла в задании)

In [2]:
import wikipedia
import numpy as np
import os
import json
import requests
import math

In [19]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [20]:
wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

c:\program files (x86)\python35-32\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\program files (x86)\python35-32\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Дивергент (айрық)
kk 99
Skipping page Антопіль
Skipping page Самохвалов
Skipping page Амброз
Skipping page Бельангревіль
Skipping page Житниця
uk 95
Skipping page Мастэрс 2012 (снукер)
Skipping page Шаўчэнка
Skipping page Жалезнагорск
Skipping page Сіманаўка
Skipping page Ідолта
Skipping page Авадзі
Skipping page Струга (значэнні)
be 93
Skipping page Pierre Lacour
Skipping page Louis de Weck
Skipping page Kalvarija
fr 97


In [21]:
print(wiki_texts['kk'][0])

Сатовча (Благоевград облысы)
Сатовча (болг. Сатовча) — Болгариядағы ауыл. Благоевград облысына қарасты Сатовча қауымының құрамына кіреді.
2010 жылғы мәліметтер бойынша тұрғындарының саны 2 026 адамды құрайды.
Пошта индексі — 2950. Телефон коды — 07541. ЕКАТТЕ коды — 65440.


 Картада 
bgmaps.com электронды картада орналасуы
emaps.bg электронды картада орналасуы
Google электронды картада орналасуы


 Сыртқы сілтемелер 
Тұрғын статистикасы  (болг.)


## Словарный метод

Теперь сделаем частотный список слов, например, для языка қазақша

In [22]:
import codecs
import collections
import sys

In [23]:
def tokenize(text):
    return text.split(' ')

def freqs_dict(language_code):
    
    freqs = collections.defaultdict(lambda: 0)

    corpus = wiki_texts[language_code]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs[word] += 1
            
    freqs_dict = {}
            
    for word in sorted(freqs, key=lambda w: freqs[w], reverse=True):
        freqs_dict[word] = freqs[word]
    
    return freqs_dict

In [24]:
print(freqs_dict('kk'))

{'': 110, 'козлаю': 1, 'дурлангендурланген': 1, 'тыс': 1, '032.': 1, 'басқарудың': 1, 'нормандия': 1, 'беларусь': 1, 'атақтардың': 1, 'жігіт': 1, 'батыр...»': 1, 'bureau.үлгі:миннесота': 1, 'аудандарында': 2, 'өздері': 1, '30-дан': 1, 'бұршағы': 1, 'сор': 1, 'онда': 1, 'қатысып,': 1, 'жылдың': 14, 'хат”,': 1, 'салалардағы': 1, 'меңгерушісі.': 1, 'күшті': 2, 'есіл': 2, 'құрамына': 28, 'маршрутты': 1, 'қасқаөгіз': 1, 'ерте': 2, '↑': 1, 'аяқталды.': 1, 'census': 7, 'жеріне': 4, 'тұрқы': 1, 'тамағы': 1, 'жайық,': 1, 'қалдырып': 1, 'кірген.': 2, 'мөлшерде': 2, 'континенттерден': 1, 'жоспарлы': 1, 'жиынтығы.': 2, 'қажының': 1, '(1965).': 1, 'қатысады.сонымен': 1, 'waveland)': 1, 'саздарағартқыш': 1, '432125электронды': 1, 'әйел': 1, 'байқалады.': 1, 'бағдарламасына': 1, 'тавда': 1, 'өртеп,': 1, 'сүйгіш,': 1, 'тексерілді,': 1, '1000,': 1, 'айтар': 1, 'ұйымдастырушы': 1, '(бейресми': 1, 'ішкі': 1, 'алғызады.': 1, '(archive)': 1, 'сіңірген': 1, 'қаз,': 1, 'покарисапокариса': 1, 'бөлінуі': 1, 'к

Все работает, но теперь посчитаем частоту и возьмем от нее логарифм для удобства, как в этой статье: http://practicalcryptography.com/miscellaneous/machine-learning/tutorial-automatic-language-identification-word-ba/

Усовершенсвуем предыдущую функцию

In [25]:
def freqs_dict1(language_code):
    
    freqs = collections.defaultdict(lambda: 0)

    corpus = wiki_texts[language_code]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs[word] += 1
            
    freqs_dict = {}
            
    for word in sorted(freqs, key=lambda w: freqs[w], reverse=True):
        freqs_dict[word] = math.log(freqs[word] / len(freqs))
    
    return freqs_dict

In [26]:
print(freqs_dict1('kk'))

{'': -3.7376696182833684, 'козлаю': -8.438149984075784, 'дурлангендурланген': -8.438149984075784, 'тыс': -8.438149984075784, '032.': -8.438149984075784, 'басқарудың': -8.438149984075784, 'нормандия': -8.438149984075784, 'беларусь': -8.438149984075784, 'атақтардың': -8.438149984075784, 'жігіт': -8.438149984075784, 'батыр...»': -8.438149984075784, 'bureau.үлгі:миннесота': -8.438149984075784, 'аудандарында': -7.745002803515839, 'өздері': -8.438149984075784, '30-дан': -8.438149984075784, 'бұршағы': -8.438149984075784, 'сор': -8.438149984075784, 'онда': -8.438149984075784, 'қатысып,': -8.438149984075784, 'жылдың': -5.799092654460526, 'хат”,': -8.438149984075784, 'салалардағы': -8.438149984075784, 'меңгерушісі.': -8.438149984075784, 'күшті': -7.745002803515839, 'есіл': -7.745002803515839, 'құрамына': -5.10594547390058, 'маршрутты': -8.438149984075784, 'қасқаөгіз': -8.438149984075784, 'ерте': -7.745002803515839, '↑': -8.438149984075784, 'аяқталды.': -8.438149984075784, 'census': -6.4922398350

Посмотрим какие вообще языки есть в википедии

In [9]:
code2lang = wikipedia.languages()

In [48]:
code2lang.keys()

dict_keys(['krl', 'bm', 'mi', 'din', 'hrx', 'sdc', 'su', 'bg', 'nan', 'sq', 'bxr', 'jut', 'nov', 'tet', 'kk-arab', 'lus', 'tcy', 'map-bms', 'zea', 'frc', 'cdo', 'de-at', 'sat', 'gag', 'sah', 'ks', 'nrm', 'hu-formal', 'ku', 'pms', 'loz', 'pl', 'rmy', 'pa', 'dty', 'sgs', 'mhr', 'grc', 'frr', 'nn', 'cbk-zam', 'km', 'mt', 'zgh', 'pcd', 'sli', 'new', 'lmo', 'uz', 'bpy', 'yo', 'roa-tara', 'gd', 'zh-hk', 'rup', 'or', 'ru', 'ch', 'he', 'nv', 'tum', 'ha', 'pdc', 'be-tarask', 'nds-nl', 'av', 'wa', 'af', 'aln', 'dtp', 'jbo', 'kl', 'tzm', 'la', 'kk-latn', 'rgn', 'gan', 'bjn', 'ase', 'als', 'scn', 'luz', 'de', 'nap', 'kbd-cyrl', 'gcr', 'uk', 'ta', 'en-gb', 'hz', 'ii', 'bcl', 'glk', 'gv', 'olo', 'gor', 'xal', 'lfn', 'cy', 'vi', 'sn', 'skr-arab', 'sma', 'eu', 'arz', 'mr', 'dz', 'kab', 'nl', 'bar', 'lrc', 'fa', 'ia', 'nds', 'iu', 'stq', 'bi', 'sa', 'el', 'liv', 'sco', 'id', 'ny', 'fur', 'ext', 'cv', 'nb', 'crh', 'bo', 'vep', 'ps', 'ff', 'eml', 'jv', 'zh', 'zh-yue', 'khw', 'zh-tw', 'ik', 've', 'fo', 't

In [59]:
print(freqs_dict1('cho'))

KeyError: 'cho'

Почему-то тексты есть не для всех этих языков, а только для четырех...

In [27]:
languages_fr = {}
for language in code2lang.keys():
    try:
        languages_fr[language] = freqs_dict1(language)
    except KeyError:
        continue
print(languages_fr.keys())

dict_keys(['uk', 'kk', 'be', 'fr'])


Сделаем частотные словари для каждого из этих языков без повторений

In [36]:
def predict_language(text, languages_fr):
    t = open(text, 'r', encoding='utf-8').read()
    
    d = {}
    i = 0
    
    for language in languages_fr.values():
        sum_lang = 0
        for word in t:
            if word in language.keys():
                sum_lang += language[word]
        d[list(languages_fr.keys())[i]] = sum_lang
        i += 1
    print(d)
    
    for key in d:
        if d[key] == max(list(d.values())):
            return key
        

In [29]:
predict_language('english_text.txt', languages_fr)

{'uk': -173.20499480756402, 'kk': -157.14939664879202, 'be': -140.2666619603087, 'fr': -283.4121869505448}


'be'

Этот метод нам плохо предсказал язык. 

In [37]:
predict_language('french_text.txt', languages_fr)

{'uk': -915.5731973048346, 'kk': -846.5149498940389, 'be': -895.0455917794976, 'fr': -1358.8085679708545}


'kk'

## Второй метод. Метод нграмм

In [30]:
from itertools import islice, tee
from collections import Counter, defaultdict
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)

In [31]:
def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return set(ngrams)

In [32]:
# посчитаем частотность нграммов
lang2char_ngrams_freqs = defaultdict(Counter)
table = str.maketrans({ch: None for ch in punctuation})

for lang in wiki_texts:
    for text in wiki_texts[lang]:
        char_ngrams = make_ngrams(text.lower().strip(' ').translate(table))
        lang2char_ngrams_freqs[lang].update(char_ngrams)

# сделаем множества с топ 300 нграммамим

lang2char_ngrams = {}
for lang in lang2char_ngrams_freqs:
    topn = [word for word, freq in lang2char_ngrams_freqs[lang].most_common(300)]
    lang2char_ngrams[lang] = set(topn)

In [33]:
def predict_language(text, lang2char):
    text_ngrams = make_ngrams(text.lower().translate(table))
    
    lang2sim = {}
    
    for lang in lang2char:
        z = (text_ngrams - punctuation)
        intersect = len(z & lang2char[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

In [34]:
predict_language('What is going on? Does anybody know this?', lang2char_ngrams)

'fr'

In [35]:
predict_language('On ne le sait pas toujours, mais les grands artisans chocolatiers sont souvent basés en France. Un cacao qui joue la carte du 100% cocorico, aussi bon que beau.', lang2char_ngrams)

'fr'

## Вывод

На мой взгляд этот метод работает лучше, потому что другие три языка в выборке там вообще на кириллице, поэтому лейбл французский (из-за алфавита) гораздо больше подходит к английскому языку, чем be.

Также в первом методе даже тескт на французском, на языке, который есть в выборке, определяется непраивльно. А во втором методе текст на французском языке определяется неправильно.